In [1]:
import random

from tqdm import tqdm

import torch
import torch.nn.functional as F
from   torch.utils.data import Dataset, DataLoader
from   torch import nn

## Prepare Data
 - Load
 - Convert characters to ints
 - Convert words -> characters -> ints -> tensors
 - Split into 80% training 20% validation

In [2]:
# https://github.com/MK2112/nn-zero-to-hero-notes/blob/main/names.txt

with open('./names.txt', 'r') as f:
    words = f.read().splitlines()
    
words[0:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
# copied from the video
chars = sorted(list(set(''.join(words))))
stoi  = {s:i+1 for i,s in enumerate(chars) }
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
def w2ten(w = 'emma'):
    pass

w = 'emma'
w = f'.{w}.'
w = [stoi[e] for e in w] 
w = [[i,j] for i,j in zip(w, w[1:])]
torch.tensor(w)

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1],
        [ 1,  0]])

In [5]:
def w2ten(w = 'emma'):
    w = f'.{w}.'
    w = [stoi[e] for e in w] 
    w = [[i,j] for i,j in zip(w, w[1:])]
    return torch.tensor(w)

In [6]:
# optional way to visualize the tensor
# import numpy as np
# import matplotlib.pyplot as plt
# plt.imshow(w2ten().numpy())

In [7]:
random.seed(42)
random.shuffle(words)
words[0:10]

['yuheng',
 'diondre',
 'xavien',
 'jori',
 'juanluis',
 'erandi',
 'phia',
 'samatha',
 'phoenix',
 'emmelynn']

In [8]:
trn_cutoff = int(len(words) * .8)

trn_bigram = torch.concat([w2ten(w = w) for w in tqdm(words[0:trn_cutoff])])
val_bigram = torch.concat([w2ten(w = w) for w in tqdm(words[trn_cutoff:len(words)])])

100%|██████████| 6407/6407 [00:00<00:00, 190642.07it/s]


## Create training and validation dataloaders

In [9]:
class ds(Dataset):
    def __init__(self, c0, c1):
        self.c0, self.c1 = c0, c1
    
    def __len__(self):
        return len(self.c0)
    
    def __getitem__(self, idx):
        return self.c0[idx], self.c1[idx]

In [10]:
# create training /validation dataloaders

# boolean for if a gpu is available
gpu = torch.cuda.is_available()


trn_dl = DataLoader(
    dataset = ds(                    # A if bool else B    move data to the gpu if a gpu is available.
        c0 = trn_bigram[:, 0].to('cuda') if gpu else trn_bigram[:, 0],
        c1 = trn_bigram[:, 1].to('cuda') if gpu else trn_bigram[:, 1]
        ),
    batch_size = 256,
    shuffle = False
)



val_dl = DataLoader(
    dataset = ds(
        c0 = val_bigram[:, 0].to('cuda') if gpu else val_bigram[:, 0],
        c1 = val_bigram[:, 1].to('cuda') if gpu else val_bigram[:, 1]
        ),
    batch_size = 256,
    shuffle = False
)

In [11]:
# Transform int -> one hot encoding 
(val_bigram[0:5, 0],
 F.one_hot(val_bigram[0:5, 0], num_classes=27).float()
)

(tensor([ 0,  1, 13,  1, 25]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0.]]))

## Work through one prediction

In [12]:
# Define a model to accept one hot encoding and produce probability (as logits) of next character
class BGM(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(27, 27)
        )
    
    def forward(self, xi):
        logits = self.layers(xi)
        return logits

In [13]:
n = BGM().to('cuda') if gpu else BGM()

In [14]:
c0, c1 = next(iter(trn_dl))
c0 = F.one_hot(c0, num_classes=27).float()

In [15]:
logits = n(c0)
logits

tensor([[-0.0707,  0.0741,  0.0065,  ...,  0.0684, -0.0427, -0.2139],
        [-0.2454,  0.2792, -0.3072,  ...,  0.0822, -0.0525,  0.0209],
        [-0.2111,  0.2480, -0.3311,  ...,  0.0175, -0.1302, -0.0771],
        ...,
        [-0.1746,  0.1038,  0.0441,  ...,  0.0031, -0.0874, -0.1504],
        [-0.2629,  0.0434, -0.2971,  ...,  0.1090, -0.1385, -0.2107],
        [-0.1018, -0.1033, -0.0805,  ...,  0.1053,  0.0799, -0.0821]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
# calculate the loss for this minibatch
loss_fn = nn.CrossEntropyLoss()
loss_fn(logits, F.one_hot(c1, num_classes=27).float())

tensor(3.3180, device='cuda:0', grad_fn=<DivBackward1>)

In [17]:
# Convert logits to probabilities
pred_probab = F.softmax(logits, dim=1)
pred_probab

tensor([[0.0369, 0.0426, 0.0398,  ..., 0.0424, 0.0379, 0.0319],
        [0.0308, 0.0521, 0.0290,  ..., 0.0428, 0.0374, 0.0402],
        [0.0315, 0.0499, 0.0279,  ..., 0.0396, 0.0342, 0.0360],
        ...,
        [0.0332, 0.0438, 0.0413,  ..., 0.0396, 0.0362, 0.0340],
        [0.0297, 0.0404, 0.0287,  ..., 0.0431, 0.0336, 0.0313],
        [0.0340, 0.0340, 0.0348,  ..., 0.0419, 0.0408, 0.0347]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [18]:
# get the most likely next character
pred_probab.argmax(1)

tensor([10,  1, 13,  1, 13, 13, 13, 10, 10,  4, 13, 13, 10, 24, 13, 10, 10, 10,
        10,  4, 13, 13, 10, 13, 13, 24,  4, 10, 13, 13, 10, 13, 13, 13,  4, 17,
        10, 13, 24, 10, 13, 10,  4, 10, 13,  1,  4, 10, 10, 17, 10, 17, 10, 13,
         1, 10, 10, 13,  1, 13, 13, 13,  4, 10, 10, 13, 17, 17, 13, 13,  1, 13,
        13, 10,  1, 13, 13, 13, 10, 13, 10,  1, 13, 13, 13,  4, 17, 10,  4, 10,
        13, 13, 10, 13,  4, 13,  1, 10, 10, 10, 13, 13, 10,  1, 10, 13, 10, 13,
        13, 17, 13, 13,  1,  1, 13, 13, 10,  1, 13, 13, 10, 13, 13, 10, 24, 10,
         1, 13, 13, 10, 24, 10, 13, 13, 10, 24, 10,  4, 13, 13, 10,  4, 13, 10,
        13, 10, 10, 13, 13, 13, 10, 10, 24,  4, 10, 13, 13,  1, 13, 13, 13, 10,
        24, 13, 10, 10, 24,  4, 10, 13, 13, 13, 13, 13, 10, 10, 17, 13,  4,  4,
        10, 10, 13, 13, 10, 13, 10, 13, 13, 17, 13,  1,  4, 13, 10, 13, 10, 13,
         4,  1, 10,  1, 10,  1, 10,  1, 13, 13,  1, 10, 10, 13, 13, 13, 10,  1,
        10, 24,  1, 17, 10, 13, 24, 13, 

## Define training behavior for one epoch and train

In [19]:
def train_epoch(dataloader, model, loss_fn, optimizer, return_loss):
    model.train()
    for i, (c0, c1) in enumerate(dataloader):
        c0 = F.one_hot(c0, num_classes=27).float()
        c1 = F.one_hot(c1, num_classes=27).float()

        logits = n(c0)
        loss = loss_fn(logits, c1)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if return_loss:
            return loss.item()

In [20]:
# define bigram model
n = BGM().to('cuda') if gpu else BGM()

# set up optimizer
opt = torch.optim.SGD(n.parameters(), lr=1e-3)

# test training
train_epoch(dataloader = trn_dl, model = n, loss_fn = nn.CrossEntropyLoss(), optimizer = opt, return_loss = True)

3.33035945892334

In [21]:
print('epoch\tloss')
print('-----\t------------------')
for k in range(200):
    _ = train_epoch(dataloader = trn_dl, model = n, loss_fn = nn.CrossEntropyLoss(), optimizer = opt, return_loss = True)
    if (k % 10) == 0:
        print(f'{k}\t{_}')

epoch	loss
-----	------------------
0	3.3303134441375732
10	3.329852819442749
20	3.329392433166504
30	3.328932762145996
40	3.3284730911254883
50	3.3280134201049805
60	3.32755446434021
70	3.3270959854125977
80	3.3266375064849854
90	3.326179265975952
100	3.325721263885498
110	3.3252639770507812
120	3.3248066902160645
130	3.324349880218506
140	3.3238933086395264
150	3.323436975479126
160	3.3229808807373047
170	3.3225255012512207
180	3.3220701217651367
190	3.321615219116211
